In [3]:
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import date, timedelta, datetime
import pandas as pd
import threading
import bisect

In [4]:
today = date.today()
day_delta=90

#today = today - timedelta(day_delta)

start = today - timedelta(day_delta)
start= start.strftime("%Y-%m-%d")

end = today + timedelta(1)
end = end.strftime("%Y-%m-%d")
print(start,end)

2020-08-01 2020-10-31


In [3]:
    ticker='SE'
    today = datetime.today()
    days45 = today + timedelta(45)

    optTicker = yf.Ticker(ticker)
    last_price=optTicker.history().tail(1)['Close'].iloc[0]

    date_list=optTicker.options
    date_df = pd.to_datetime(date_list)
    date_diffs = abs(days45 - date_df)
    closest_date = date_list[date_diffs.argmin()]


    opt = optTicker.option_chain(closest_date)

    optCalls=opt[0]
    optPuts=opt[1]

    strikes=optPuts['strike'].tolist()
    strikes.sort()
    
    index = bisect.bisect(strikes, last_price)
    sellPut=strikes[index-1]
    if sellPut-5 in strikes:
        buyPut=sellPut-5
    else:
        index = bisect.bisect(strikes, sellPut-1.5)
        buyPut=strikes[index-1]

    sellDF=optPuts[optPuts['strike']==sellPut]
    buyDF=optPuts[optPuts['strike']==buyPut]

    width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
    premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
    if premium<0:
        premium=(sellDF.bid.iloc[0]+sellDF.ask.iloc[0])/2-(buyDF.bid.iloc[0]+buyDF.ask.iloc[0])/2
    if premium>width:
        premium=0
        IV=0
    else:
        IV=sellDF.impliedVolatility.iloc[0]
#     index = bisect.bisect(strikes, last_price)
#     sellPut=strikes[index-1]
#     if sellPut-5 in strikes:
#         buyPut=sellPut-5
#     else:
#         index = bisect.bisect(strikes, sellPut-1.5)
#         buyPut=strikes[index-1]

#     sellDF=optPuts[optPuts['strike']==sellPut]
#     buyDF=optPuts[optPuts['strike']==buyPut]

#     width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
#     premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
#     IV=sellDF.impliedVolatility.iloc[0]
    #print(sellPut,buyPut)
    #optPuts.head()
    #print([width,premium,IV])
    #optPuts[(optPuts['strike'] ==67.5) | (optPuts['strike']==62.5) ].head()
    #optPuts.head()

In [5]:
ticker='MSFT'
ahTicker = yf.Ticker(ticker)
ahTicker.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-09-30,207.729996,211.979996,206.539993,210.330002,33780700,0,0
2020-10-01,213.490005,213.990005,211.320007,212.460007,27158400,0,0
2020-10-02,208.000000,210.990005,205.539993,206.190002,33154800,0,0
2020-10-05,207.220001,210.410004,206.979996,210.380005,21331600,0,0
2020-10-06,208.820007,210.179993,204.820007,205.910004,28554300,0,0
2020-10-07,207.059998,210.110001,206.720001,209.830002,25681100,0,0
2020-10-08,210.509995,211.190002,208.320007,210.580002,19925800,0,0
2020-10-09,211.229996,215.860001,211.229996,215.809998,26458000,0,0
2020-10-12,218.789993,223.860001,216.809998,221.399994,40461400,0,0


In [5]:
def getPutSpread(last_price,ticker):
    today = datetime.today()
    days45 = today + timedelta(45)

    optTicker = yf.Ticker(ticker)
    #last_price=optTicker.history().tail(1)['Close'].iloc[0]

    date_list=optTicker.options
    date_df = pd.to_datetime(date_list)
    date_diffs = abs(days45 - date_df)
    closest_date = date_list[date_diffs.argmin()]


    opt = optTicker.option_chain(closest_date)

    optCalls=opt[0]
    optPuts=opt[1]

    strikes=optPuts['strike'].tolist()
    strikes.sort()

    index = bisect.bisect(strikes, last_price)
    sellPut=strikes[index-1]
    if sellPut-5 in strikes:
        buyPut=sellPut-5
    else:
        index = bisect.bisect(strikes, sellPut-1.5)
        buyPut=strikes[index-1]

    sellDF=optPuts[optPuts['strike']==sellPut]
    buyDF=optPuts[optPuts['strike']==buyPut]

    width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
    premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
    if premium<0:
        premium=(sellDF.bid.iloc[0]+sellDF.ask.iloc[0])/2-(buyDF.bid.iloc[0]+buyDF.ask.iloc[0])/2
    if premium>width:
        premium=0
        IV=0
    else:
        IV=sellDF.impliedVolatility.iloc[0]
    outputArr=[width,premium,IV]
    return outputArr
    #print(optPuts.columns)
    #print (width,premium,IV)




In [6]:
def stok_data_pull():
        dfOutput=pd.DataFrame()

        tickers=['MA','ADBE','TSLA','LYFT','PANW','ESTC','SPG','FB','JMIA','TWLO','LULU','LMND','SAGE','ETSY','LRCX','NET','WORK','NFLX','MTB','DKNG','EQIX','EB','PSTH','ARKK','IPOB','TJX','QDEL','DOW','DOCU','CVS','PENN','MGM','MSFT','AMZN','GOOGL','WBA','GDDY','F','BA','GPS','VIAC','MDP','HUYA','ICLN','MJ','PTON','WING','OTRK','FSLY','MELI','SE','CSCO','NNOX','ASAN','AVGO','QCOM','WFC','TEAM','ISRG','DPZ','PZZA','ROKU','COF','TDOC','CPRI','GRWG','CRM','TLRY','GLD','SLV','RKT','MDB','TEAM','RDFN','UPS','SQ','LOGI','AMWL','FVRR','TLRY','CRWD','TTD','DDOG','PLAY','U','AMT','MTCH','SDGR','RHP','JPM','SPCE','TTWO','DXCM']

        for t in tickers:
            df = yf.download(t.replace('.', '-').replace('^', '-'), 
                                  start=start, 
                                  end=end,
                                  prepost=True,
                                  progress=False)

            df['Stock']=t
            dfTemp=df.reset_index()
            last_price=dfTemp[dfTemp['Date']==dfTemp['Date'].max()].Close.iloc[0]
            try:
                putSpread=getPutSpread(last_price,t)
            except:
                putSpread=[0,0,0]
            df['putWidth']=putSpread[0]
            df['putPremium']=putSpread[1]
            df['putIV']=putSpread[2]
            try:
                dfOutput=pd.concat([dfOutput, df])
            except:
                print('error '+t)

        #dfOutput.head(100)
        dfOutput['dt']=''
        dfOutput['dt'].iloc[0] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        dfOutput.to_csv('data.csv')
        return "done"

In [7]:
#one time

stok_data_pull()

'done'

In [8]:
def get_stock_data():
    threading.Timer(300.0, get_stock_data).start() # called every 5 minute
    try:
        stok_data_pull()
    except:
        pass

get_stock_data()